# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who:
   - have a **low total_claim_amount** (e.g., below $1,000),
   - have a response "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze:
   - the average `monthly_premium` and/or customer lifetime value by `policy_type` and `gender` for customers who responded "Yes", and
   - compare these insights to `total_claim_amount` patterns, and discuss which segments appear most profitable or low-risk for the company.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"

df = pd.read_csv(url)


# Función de limpieza de nombres de columnas
def clean_column_names(dataframe):
    cols = dataframe.columns
    new_cols = []
    for col in cols:
        new_col = col.lower().replace(' ', '_')
        new_cols.append(new_col)
    dataframe.columns = new_cols
    return dataframe

# Limpiar los nombres de las columnas del DataFrame
df_clean = clean_column_names(df.copy())

print("\nNombres de columnas limpiados (primeras 5):")
print(df_clean.head())


Nombres de columnas limpiados (primeras 5):
   unnamed:_0 customer       state  customer_lifetime_value response  \
0           0  DK49336     Arizona              4809.216960       No   
1           1  KX64629  California              2228.525238       No   
2           2  LZ68649  Washington             14947.917300       No   
3           3  XL78013      Oregon             22332.439460      Yes   
4           4  QA50777      Oregon              9025.067525       No   

   coverage education effective_to_date employmentstatus gender  ...  \
0     Basic   College           2/18/11         Employed      M  ...   
1     Basic   College           1/18/11       Unemployed      F  ...   
2     Basic  Bachelor           2/10/11         Employed      M  ...   
3  Extended   College           1/11/11         Employed      M  ...   
4   Premium  Bachelor           1/17/11    Medical Leave      F  ...   

   number_of_open_complaints number_of_policies     policy_type        policy  \
0       

In [22]:

print(df_clean.columns)



Index(['unnamed:_0', 'customer', 'state', 'customer_lifetime_value',
       'response', 'coverage', 'education', 'effective_to_date',
       'employmentstatus', 'gender', 'income', 'location_code',
       'marital_status', 'monthly_premium_auto', 'months_since_last_claim',
       'months_since_policy_inception', 'number_of_open_complaints',
       'number_of_policies', 'policy_type', 'policy', 'renew_offer_type',
       'sales_channel', 'total_claim_amount', 'vehicle_class', 'vehicle_size',
       'vehicle_type'],
      dtype='object')


array([9, 1, 2, 7, 4, 3, 6, 8, 5])

In [2]:
# Crear un nuevo DataFrame con clientes específicos

# Definir el umbral de reclamo bajo
umbral_reclamo = 1000

# Filtrar a los clientes: reclamo bajo Y respuesta "Yes"
df_clientes_filtrados = df_clean[
    (df_clean['total_claim_amount'] < umbral_reclamo) &
    (df_clean['response'] == 'Yes')
]

print("\n--- DataFrame de Clientes Filtrados (Reclamo < $1000 y Respuesta 'Yes') ---")
print(f"Total de clientes filtrados: {df_clientes_filtrados.shape[0]}")
print(df_clientes_filtrados[['total_claim_amount', 'response']].head())


--- DataFrame de Clientes Filtrados (Reclamo < $1000 y Respuesta 'Yes') ---
Total de clientes filtrados: 1399
    total_claim_amount response
3           484.013411      Yes
8           739.200000      Yes
15          547.200000      Yes
19           19.575683      Yes
27           60.036683      Yes


In [6]:
# Análisis de clientes que respondieron "Yes"

# Filtrar solo a los clientes con respuesta "Yes"
df_yes = df_clean[df_clean['response'] == 'Yes']

# Agrupar por policy_type y gender, y calcular el promedio
df_analisis_yes = df_yes.groupby(['policy_type', 'gender']).agg(
    avg_monthly_premium_auto=('monthly_premium_auto', 'mean'),
    avg_clv=('customer_lifetime_value', 'mean'),
    avg_claim=('total_claim_amount', 'mean'),
    count=('customer', 'count') # Contamos el número de clientes en cada segmento
).reset_index()

# Redondear las columnas numéricas para una mejor visualización
df_analisis_yes = df_analisis_yes.round(2)

print("\n--- Análisis por Policy Type y Gender (Clientes con Respuesta 'Yes') ---")
print(df_analisis_yes)
print("\n**Conclusión sobre Rentabilidad/Riesgo:**")
print("> Un segmento es potencialmente más 'rentable' si tiene un alto CLV/Prima promedio, pero un 'bajo' reclamo promedio (avg_claim).")
print("> Busque segmentos con ALTO 'avg_clv' y BAJO 'avg_claim'.")


--- Análisis por Policy Type y Gender (Clientes con Respuesta 'Yes') ---
      policy_type gender  avg_monthly_premium_auto  avg_clv  avg_claim  count
0  Corporate Auto      F                     94.30  7712.63     433.74    169
1  Corporate Auto      M                     92.19  7944.47     408.58    154
2   Personal Auto      F                     99.00  8339.79     452.97    540
3   Personal Auto      M                     91.09  7448.38     457.01    536
4    Special Auto      F                     92.31  7691.58     453.28     35
5    Special Auto      M                     86.34  8247.09     429.53     32

**Conclusión sobre Rentabilidad/Riesgo:**
> Un segmento es potencialmente más 'rentable' si tiene un alto CLV/Prima promedio, pero un 'bajo' reclamo promedio (avg_claim).
> Busque segmentos con ALTO 'avg_clv' y BAJO 'avg_claim'.


In [14]:
# Análisis de clientes por estado

# Contar el número de clientes en cada estado
df_clientes_por_estado = df_clean['state'].value_counts().reset_index()
# asignar nombres claros a las columnas nuevas despues de .reset_index() modificando el dataframe que acabamos de crear df_clientes_por_estado
df_clientes_por_estado.columns = ['state', 'total_customers']

# Definir el umbral de clientes
umbral_clientes = 500
# Filtrar los estados con más de 500 clientes
df_estados_populares = df_clientes_por_estado[df_clientes_por_estado['total_customers'] > umbral_clientes]

print(f"\n--- Estados con más de {umbral_clientes} clientes ---")
print(df_estados_populares)


--- Estados con más de 500 clientes ---
        state  total_customers
0  California             3552
1      Oregon             2909
2     Arizona             1937
3      Nevada              993
4  Washington              888


In [15]:
# Máximo, mínimo y mediana de CLV por Educación y Género

df_clv_por_segmento = df_clean.groupby(['education', 'gender']).agg(
    max_clv=('customer_lifetime_value', 'max'),
    min_clv=('customer_lifetime_value', 'min'),
    median_clv=('customer_lifetime_value', 'median')
).reset_index()

# Redondear las columnas numéricas para una mejor visualización
df_clv_por_segmento = df_clv_por_segmento.round(2)

print("\n--- Estadísticas de Customer Lifetime Value (CLV) por Educación y Género ---")
print(df_clv_por_segmento)

print("\n**Conclusión sobre CLV:**")
print("> La 'median_clv' ofrece una visión más robusta del CLV 'típico' para cada segmento que el promedio, ya que es menos sensible a los valores atípicos.")
print("> Un segmento con una 'median_clv' alta podría ser un objetivo valioso para futuras campañas.")


--- Estadísticas de Customer Lifetime Value (CLV) por Educación y Género ---
              education gender   max_clv  min_clv  median_clv
0              Bachelor      F  73225.96  1904.00     5640.51
1              Bachelor      M  67907.27  1898.01     5548.03
2               College      F  61850.19  1898.68     5623.61
3               College      M  61134.68  1918.12     6005.85
4                Doctor      F  44856.11  2395.57     5332.46
5                Doctor      M  32677.34  2267.60     5577.67
6  High School or Below      F  55277.45  2144.92     6039.55
7  High School or Below      M  83325.38  1940.98     6286.73
8                Master      F  51016.07  2417.78     5729.86
9                Master      M  50568.26  2272.31     5579.10

**Conclusión sobre CLV:**
> La 'median_clv' ofrece una visión más robusta del CLV 'típico' para cada segmento que el promedio, ya que es menos sensible a los valores atípicos.
> Un segmento con una 'median_clv' alta podría ser un objetivo 

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [24]:
# Usamos 'effective_to_date' como sustituto de 'policy_issue_date'
df_clean['effective_to_date'] = pd.to_datetime(df_clean['effective_to_date'], errors='coerce')

# Extraer el nombre del mes
df_clean['month'] = df_clean['effective_to_date'].dt.strftime('%B')

# Definir el orden correcto de los meses (para pivotar correctamente)
month_order = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 'December']
df_clean['month'] = pd.Categorical(df_clean['month'], categories=month_order, ordered=True)

# Crear la tabla pivotante (Continúa igual)
df_policies_by_month_state = pd.pivot_table(
    df_clean,
    index='state',          
    columns='month',        
    values='customer',      
    aggfunc='count'         
).fillna(0)
print("--- 📚 Pólizas Vendidas por Estado y Mes (Tabla Pivotante Completa) ---")
print(df_policies_by_month_state.head())

--- 📚 Pólizas Vendidas por Estado y Mes (Tabla Pivotante Completa) ---
month       January  February  March  April  May  June  July  August  \
state                                                                  
Arizona        1008       929      0      0    0     0     0       0   
California     1918      1634      0      0    0     0     0       0   
Nevada          551       442      0      0    0     0     0       0   
Oregon         1565      1344      0      0    0     0     0       0   
Washington      463       425      0      0    0     0     0       0   

month       September  October  November  December  
state                                               
Arizona             0        0         0         0  
California          0        0         0         0  
Nevada              0        0         0         0  
Oregon              0        0         0         0  
Washington          0        0         0         0  


C:\Users\Marta\AppData\Local\Temp\ipykernel_7440\270360997.py:13: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  df_policies_by_month_state = pd.pivot_table(


In [25]:
# 1. Agrupar por estado y contar el total de pólizas
df_total_policies_by_state = df_clean.groupby('state')['customer'].count()

# 2. Ordenar y seleccionar los 3 estados principales
top_3_states = df_total_policies_by_state.sort_values(ascending=False).head(3).index.tolist()

print(f"\n--- Top 3 Estados con Mayor Número de Pólizas Vendidas ---")
print(top_3_states)


--- Top 3 Estados con Mayor Número de Pólizas Vendidas ---
['California', 'Oregon', 'Arizona']


In [26]:
## DataFrame del Top 3 de Estados

df_top_3_policies_by_month_state = df_policies_by_month_state.loc[top_3_states]

print("\n--- 🏆 Pólizas Vendidas por Mes para el Top 3 de Estados ---")
print(df_top_3_policies_by_month_state)


--- 🏆 Pólizas Vendidas por Mes para el Top 3 de Estados ---
month       January  February  March  April  May  June  July  August  \
state                                                                  
California     1918      1634      0      0    0     0     0       0   
Oregon         1565      1344      0      0    0     0     0       0   
Arizona        1008       929      0      0    0     0     0       0   

month       September  October  November  December  
state                                               
California          0        0         0         0  
Oregon              0        0         0         0  
Arizona             0        0         0         0  
